In [1]:
import pandas as pd
import numpy as np 

pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', 50)
pd.set_option('display.float_format', '{:.2f}'.format)

df = pd.read_excel('DT1_PAINEL_COVIDBR_20200512.xlsx')

In [3]:
# df
df[df.municipio =='Uberlândia'].head()

,regiao,estado,municipio,coduf,codmun,codRegiaoSaude,nomeRegiaoSaude,data,semanaEpi,populacaoTCU2019,casosAcumulado,obitosAcumulado,Recuperadosnovos,emAcompanhamentoNovos
39878,Sudeste,MG,Uberlândia,31,317020.00,31073.00,Uberlândia Araguari,2020-03-28,13,691305.00,7,0,nan,nan
39879,Sudeste,MG,Uberlândia,31,317020.00,31073.00,Uberlândia Araguari,2020-03-29,14,691305.00,7,0,nan,nan
39880,Sudeste,MG,Uberlândia,31,317020.00,31073.00,Uberlândia Araguari,2020-03-30,14,691305.00,10,0,nan,nan
39881,Sudeste,MG,Uberlândia,31,317020.00,31073.00,Uberlândia Araguari,2020-03-31,14,691305.00,10,0,nan,nan
39882,Sudeste,MG,Uberlândia,31,317020.00,31073.00,Uberlândia Araguari,2020-04-01,14,691305.00,14,0,nan,nan
